# Science Pipelines Parquet Column Review 

### SST Meeting Sept 20 2021
(Timing before DP0.2 processing not coincidence)

#### Background

The [DPDD](https://lse-163.lsst.io/) was written a long time ago and was not meant to be prescriptive in terms of column names. At some times it is over-prescriptive in its assumptions of algorithms baked into the column names. (For example, all references to multifit should be removed). The [baseline schema](https://github.com/lsst/sdm_schemas/blob/master/yml/baselineSchema.yaml). Both of these provide guidance but are not requirements.

#### SST Meeting Goals:

This meeting is to review the existing parquet tables being *output* from the pipelines during a DRP. The two users will be science users loading columns into notebooks, and DAX using them for ingest and database backup. 

Goal for today is from a science users perspective, agree on changing column names, adding, and removing columns. We all wear a lot of hats. Its OK to bring your other hats to this discussion, however my preference is that we don't speculate what another group needs. For example, DAX is responsible for the parquet ingest, and let's not pretend we know better than they do what they need. 

There are many adjacent conversations that can derail us. If I sense you heading towards these bike sheds and amusement park rides, I’ll pull you back:
* Implementation details. Fun to talk about it. Not today. 
* Schema in qserv. Also important. Not today. 

We've been slowly adding these outputs and adding to them over the years. It's time to look at them as a collective and reconcile inconsistencies. Let's record questions and items and to-do items for w_2021_40. 

#### History

A couple years ago Leanne, Jim, the pipelines folks and I went through the [object table specification](https://github.com/lsst/obs_lsst/blob/master/policy/imsim/Object.yaml), and reconciled it with the DPDD. This reconciliation resulted in a list of tickets (TODO in the Object.yaml) to remove columns from the DPDD, and make some decisions. These are on an SST epic and have been untouched: https://jira.lsstcorp.org/browse/DM-22078 Please help. 

Since then, Pipelines has added `Visit`, `CcdVisit`, `Source`, `ForcedSource`, `DiaSource`, `DiaObject`, and `ForcedSourceOnDiaObject` tables. 

Last week, the Science Pipelines team went through and requsted changes. Changes to the Parquet specs are going on `DM-31825` which I aim to merge before w_2021_40. There will be an RFC filed tomorrow Sept 21 2021, with closing date Monday, Sept 27 2021. This notebook contains the schemas on master and on DM-31825. 

#### Assumptions:

1) If we can't do QA without a column, then a scientist won't be able to understand the dataset and thus their analysis without it. We must eat our dog food. Therefore, we've errored on the side of overinclusion. 

---

### Warning, don't copy how this notebook interacts with Parquet Tables:

These are the DRP catalog outputs. Will be used for ingesting into qserv, made available on the science platform, and be used as backup for qserv. They are column stores which make them very useful for accessing millions of rows of few columns. Do not read the whole thing in if you don't have to. I do it in this notebook because I want to show you all the columns. Don't copy this idiom for code. In real life, inspect the columns and read in only the columns you want. 

#### Example of how to fetch column list and single columns:

In [1]:
import lsst.daf.butler as dafButler
import pandas as pd
pd.set_option('display.max_columns', None)

butlerRFC = dafButler.Butler('/repo/dc2', collections=['u/yusra/dc2/DM-31825'])
butler = dafButler.Butler('/repo/dc2', collections=['2.2i/runs/test-med-1/w_2021_36/DM-31699'])

columns = butlerRFC.get('sourceTable_visit.columns', visit=212071)
print(list(columns))
df = butlerRFC.get('sourceTable_visit', visit=212071, 
                   parameters={"columns": ['coord_ra', 'coord_dec', 'sky_source', 'detect_isPrimary']})
df.head()

['sourceId', 'coord_ra', 'coord_dec', 'ccdVisitId', 'parentSourceId', 'x', 'y', 'xErr', 'yErr', 'ra', 'decl', 'apFlux', 'apFluxErr', 'sky', 'skyErr', 'psfFlux', 'psfFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF', 'iyyPSF', 'ixyPSF', 'extendedness', 'localPhotoCalib', 'localPhotoCalib_flag', 'localPhotoCalibErr', 'localWcs_flag', 'localWcs_CDMatrix_2_1', 'localWcs_CDMatrix_1_1', 'localWcs_CDMatrix_1_2', 'localWcs_CDMatrix_2_2', 'blendedness_abs', 'blendedness_flag', 'blendedness_flag_noCentroid', 'blendedness_flag_noShape', 'apFlux_12_0_flag', 'apFlux_12_0_flag_apertureTruncated', 'apFlux_12_0_flag_sincCoeffsTruncated', 'apFlux_12_0_instFlux', 'apFlux_12_0_instFluxErr', 'apFlux_17_0_flag', 'apFlux_17_0_flag_apertureTruncated', 'apFlux_17_0_instFlux', 'apFlux_17_0_instFluxErr', 'apFlux_25_0_flag', 'apFlux_25_0_flag_apertureTruncated', 'apFlux_25_0_instFlux', 'apFlux_25_0_instFluxErr', 'apFlux_35_0_flag', 'apFlux_35_0_flag_apertureTruncated', 'apFlux_35_0_instFlux', 'apFlux_35_0_instFluxErr', 'a

,coord_ra,coord_dec,sky_source,detect_isPrimary
id,,,,
113854763526782977,55.720344,-36.267168,False,True
113854763526782978,55.658910,-36.171509,False,False
113854763526782979,55.657962,-36.170031,False,True
113854763526782980,55.645888,-36.150592,False,True
113854763526782981,55.691543,-36.221153,False,True


##  The Source Table


* **dimensions:** visit
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/Source.yaml
* **what**: No jointcal/FGCM for DC2 because it has a perfect reference catalog. Therefore single-frame processing is not iterative and you can assume that this is the FINAL Source Table. (This is not true for HSC, where our SourceTable is the preliminary one used for input into glocal calibration tasks)
* **primary key**:`SourceId` 
* **rows**: Contain sky sources. Be aware of flags `sky_source` and `detect_isPrimary`


**Questions:**

* We have no plan to match these to ObjectId for DP0.2. OK?
> - I thought we we never required to do this? <br>
> - I don't think we eer agreed one way or another.<br>
> - I  find it confusing that sky_sources are buried in the table. Should we split these rows into a separate table. <br>
> - Can be done after ingest. <br>
> - But parquet table names should match qserv name.<br>
> - SDSS users are used to views into tables, and would be used to a sky_source view.<br>
> - In the DPPD the Source table has an objectId with a caveat "if any"<br>
> -  hard to do in a science-goal agnostic way. <br>
> - The first step in all instructions should say: `df[df.detect_isPrimary]`<br>

 
* Need to discuss with Fritz on how to partition on Source.

**DM-31825 Changes**
* `ixxPsf` -> `ixxPSF` to match DIASourceTable. 
* Columns all start lowercase


Example:

In [2]:
df = butler.get('sourceTable_visit', visit=212071)
print("BEFORE")
print("The Source table is %s rows and %s columns" % (df.shape))
df.tail()

BEFORE
The Source table is 213182 rows and 152 columns


,sourceId,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,ApFlux,ApFluxErr,sky,skyErr,PsFlux,PsFluxErr,Ixx,Iyy,Ixy,IxxPsf,IyyPsf,IxyPsf,extendedness,LocalPhotoCalib,LocalPhotoCalib_flag,LocalPhotoCalibErr,LocalWcs_flag,LocalWcs_CDMatrix_2_1,LocalWcs_CDMatrix_1_1,LocalWcs_CDMatrix_1_2,LocalWcs_CDMatrix_2_2,Blendedness_abs,Blendedness_flag,Blendedness_flag_noCentroid,Blendedness_flag_noShape,ApFlux_12_0_flag,ApFlux_12_0_flag_apertureTruncated,ApFlux_12_0_flag_sincCoeffsTruncated,ApFlux_12_0_instFlux,ApFlux_12_0_instFluxErr,ApFlux_17_0_flag,ApFlux_17_0_flag_apertureTruncated,ApFlux_17_0_instFlux,ApFlux_17_0_instFluxErr,ApFlux_25_0_flag,ApFlux_25_0_flag_apertureTruncated,ApFlux_25_0_instFlux,ApFlux_25_0_instFluxErr,ApFlux_35_0_flag,ApFlux_35_0_flag_apertureTruncated,ApFlux_35_0_instFlux,ApFlux_35_0_instFluxErr,ApFlux_3_0_flag,ApFlux_3_0_flag_apertureTruncated,ApFlux_3_0_flag_sincCoeffsTruncated,ApFlux_3_0_instFlux,ApFlux_3_0_instFluxErr,ApFlux_4_5_flag,ApFlux_4_5_flag_apertureTruncated,ApFlux_4_5_flag_sincCoeffsTruncated,ApFlux_4_5_instFlux,ApFlux_4_5_instFluxErr,ApFlux_50_0_flag,ApFlux_50_0_flag_apertureTruncated,ApFlux_50_0_instFlux,ApFlux_50_0_instFluxErr,ApFlux_6_0_flag,ApFlux_6_0_flag_apertureTruncated,ApFlux_6_0_flag_sincCoeffsTruncated,ApFlux_6_0_instFlux,ApFlux_6_0_instFluxErr,ApFlux_70_0_flag,ApFlux_70_0_flag_apertureTruncated,ApFlux_70_0_instFlux,ApFlux_70_0_instFluxErr,ApFlux_9_0_flag,ApFlux_9_0_flag_apertureTruncated,ApFlux_9_0_flag_sincCoeffsTruncated,ApFlux_9_0_instFlux,ApFlux_9_0_instFluxErr,Extendedness_flag,FootprintArea_value,Jacobian_flag,Jacobian_value,LocalBackground_instFlux,LocalBackground_instFluxErr,LocalBackground_flag,LocalBackground_flag_noGoodPixels,LocalBackground_flag_noPsf,PixelFlags,PixelFlags_bad,PixelFlags_cr,PixelFlags_crCenter,PixelFlags_edge,PixelFlags_interpolated,PixelFlags_interpolatedCenter,PixelFlags_offimage,PixelFlags_saturated,PixelFlags_saturatedCenter,PixelFlags_suspect,PixelFlags_suspectCenter,PsfFlux_apCorr,PsfFlux_apCorrErr,PsfFlux_area,PsfFlux_flag,PsfFlux_flag_apCorr,PsfFlux_flag_edge,PsfFlux_flag_noGoodPixels,Centroid_flag,Centroid_flag_almostNoSecondDerivative,Centroid_flag_badError,Centroid_flag_edge,Centroid_flag_noSecondDerivative,Centroid_flag_notAtMaximum,Centroid_flag_resetToPeak,Variance_flag,Variance_flag_emptyFootprint,Variance_value,Calib_astrometry_used,Calib_detected,Calib_photometry_reserved,Calib_photometry_used,Calib_psf_candidate,Calib_psf_reserved,Calib_psf_used,Deblend_deblendedAsPsf,Deblend_hasStrayFlux,Deblend_masked,Deblend_nChild,Deblend_parentTooBig,Deblend_patchedTemplate,Deblend_rampedTemplate,Deblend_skipped,Deblend_tooManyPeaks,HsmPsfMoments_flag,HsmPsfMoments_flag_no_pixels,HsmPsfMoments_flag_not_contained,HsmPsfMoments_flag_parent_source,HsmShapeRegauss_flag,HsmShapeRegauss_flag_galsim,HsmShapeRegauss_flag_no_pixels,HsmShapeRegauss_flag_not_contained,HsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,band,instrument,detector,physical_filter,visit_system,visit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
113854845131163562,113854845131163562,59.586569,-35.664513,212071175,113854845131163170,338.379693,3878.645317,1.592987,1.011278,59.586569,-35.664513,2747.277916,1044.830936,1.659222,49.050772,1572.264435,327.972988,10.658975,3.525215,0.940453,2.845071,2.895513,-0.076384,1.0,0.763623,False,0.0,False,8.428151e-07,-4.787249e-07,8.425011e-07,4.785934e-07,0.167913,False,False,False,False,False,False,3597.690918,1368.255737,False,False,4187.615466,1956.389364,False,False,7812.704708,2870.876832,False,False,8061.068128,4016.119098,False,False,False,1502.660767,340.319946,False,False,False,2093.156250,508.021454,False,False,11643.502559,5731.424402,False,False,False,2890.326660,680.833496,False,False,6665.067330,8020.572474,False,False,False,3610.148682,1024.866211,False,211,False,0.998947,2.172830,64.234316,False,Fal

In [3]:
df = butlerRFC.get('sourceTable_visit', visit=212071)
print("AFTER")
print("The Source table is %s rows and %s columns" % (df.shape))
df.tail()

AFTER
The Source table is 213182 rows and 151 columns


,sourceId,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,apFlux,apFluxErr,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,extendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_flag_apertureTruncated,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,apFlux_25_0_flag,apFlux_25_0_flag_apertureTruncated,apFlux_25_0_instFlux,apFlux_25_0_instFluxErr,apFlux_35_0_flag,apFlux_35_0_flag_apertureTruncated,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,apFlux_3_0_flag,apFlux_3_0_flag_apertureTruncated,apFlux_3_0_flag_sincCoeffsTruncated,apFlux_3_0_instFlux,apFlux_3_0_instFluxErr,apFlux_4_5_flag,apFlux_4_5_flag_apertureTruncated,apFlux_4_5_flag_sincCoeffsTruncated,apFlux_4_5_instFlux,apFlux_4_5_instFluxErr,apFlux_50_0_flag,apFlux_50_0_flag_apertureTruncated,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,apFlux_6_0_flag,apFlux_6_0_flag_apertureTruncated,apFlux_6_0_flag_sincCoeffsTruncated,apFlux_6_0_instFlux,apFlux_6_0_instFluxErr,apFlux_70_0_flag,apFlux_70_0_flag_apertureTruncated,apFlux_70_0_instFlux,apFlux_70_0_instFluxErr,apFlux_9_0_flag,apFlux_9_0_flag_apertureTruncated,apFlux_9_0_flag_sincCoeffsTruncated,apFlux_9_0_instFlux,apFlux_9_0_instFluxErr,extendedness_flag,footprintArea_value,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_detected,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,band,instrument,detector,physical_filter,visit_system,visit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
113854824730068751,113854824730068751,58.376206,-35.833697,212071137,113854824730068317,2134.628100,3881.716163,0.533125,0.695611,58.376206,-35.833697,6538.214582,1052.921868,5.856143,42.545299,3178.688687,320.207954,9.139266,16.468078,9.467894,2.801028,2.814990,-0.074280,1.0,0.763623,False,0.0,False,8.489146e-07,-4.683942e-07,8.488992e-07,4.683388e-07,0.146327,False,False,False,False,False,False,8562.102539,1378.851196,False,False,12081.158851,1973.127065,False,False,19629.035948,2881.677280,False,False,29642.265668,4020.779561,False,False,False,3474.802979,342.358246,False,False,False,4383.585938,508.627228,False,False,40115.522117,5718.204755,False,False,False,5709.995605,680.440308,False,False,48872.329961,7990.414766,False,False,False,7408.836914,1026.413574,False,633,False,0.999822,7.668897,55.715089,False,False,Fa

##  The Object Table

* **dimensions:** tract 
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/Object.yaml
* **what**: The product of multi-band forced photometry (fixed centroid) and measurement. All 6 bands included, contains colors and shapes. 
* **primary key**: `objectId` 
* **rows**: Contain sky objects and patch overlaps. Be aware of flags `sky_source` and `detect_isPrimary`


This table needs the most clean up. 

2 years ago we reconciled the DPDD with with what Science Pipelines has and wants to include. This reconciliation resulted in a list of tickets (TODO in the Object.yaml): https://jira.lsstcorp.org/browse/DM-22078 Please help. 

*Questions*
* **Galaxy Colors DM-22243**
    - Proposal: Remove `grStd`, `riStd`, `izStd`, `zyStd`. Replace these with u, g, r, i, z, y certified fluxes to use for galaxy colors.Either: give it a name such as "gGalaxyColorFlux", "gModelColorFlux", "gMatchedApertureGalaxyFlux", or just advise folks to use e.g. `gGaapFlux_1_0` or `CModelFlux`
    > - If you were looking for fluxes for galaxy colors what column would you expect? 
    > - iModelFlux and rModelFlux! 
    > - Galaxy users are used to rooting around for the algo they want to use. 
    
* Is the lowercase camel case schema fixed or can we change it to snake case?
    > problem with camelcase is when it returns all caps it is unreadable. 

* Even if snake_case is a no go, making band lover case like `u_psFlux` is easier to read, and makes sharing analysis code between Source and Object tables simpler, because you can just prepend the band without changing the column name part after. 
* Prepend or append the band? (i.e. do you want columns sorted by quantity or by band?)
> - It’s not just band_algorithm vs algorithm_band, it’s band_algorithm_other_specifiers vs algorithm_other_specifiers_band, and I think the last one is particularly bad. i.e. I’m anti- kron_flux_flag_bad_shape_r. <br>
> - Then it has to be prefix.  Because putting it in the middle is very hard to automatically parse because the location depends on the column.<br>
> - Colin, you have just very effectively made the argument for prepend
* Turns out DC2 DOES have Galactic extinction. We don't have any of these columns in the Object Table. Can't add in 10 days. OK?

*TODO*
* Lauren listed columns needed by the ol' pipe_analysis here: https://jira.lsstcorp.org/browse/DM-31339 [Discussion here](https://docs.google.com/spreadsheets/d/10_zZuB0aY1lIi7hznyV-lKoACAir3ZhwC_8K0EBIEA8/edit#gid=1727863084) If we have columns that are taken from BOTH the `meas` and `forced` how do we disambiguate them by name? 



*Example:* Current pre-DM-31825/RFC version:

In [4]:
df = butler.get('objectTable_tract', tract=3828, skymap='DC2')
print("%s rows and %s columns" % (df.shape))
df.tail()

1456422 rows and 857 columns


column,tractId,x,detect_isPrimary,parentObjectId,izStd,detect_isDeblendedModelSource,coord_dec,y,xErr,xy_flag,detect_isDeblendedSource,refFwhm,refExtendedness,zyStd,merge_peak_sky,refBand,grStd,riStd,detect_fromBlend,detect_isTractInner,detect_isPatchInner,coord_ra,yErr,detect_isIsolated,objectId,patchId,skymap,tract,patch,uRa,uDecl,uPsFlux,uPsFluxErr,uBdE1,uBdE2,uBdReB,uBdReD,uBdChi2,uBdFluxB,uBdFluxBErr,uBdFluxD,uBdFluxDErr,uGaapFlux_Psf,uGaapFluxErr_Psf,uGaapFlux_0_7,uGaapFluxErr_0_7,uGaapFlux_1_0,uGaapFluxErr_1_0,uGaapFlux_Optimal,uGaapFluxErr_Optimal,uIxx,uIyy,uIxy,uIxxPsf,uIyyPsf,uIxyPsf,uKronRad,uKronFlux,uKronFluxErr,uApFlux,uApFluxErr,uAp25Flux,uAp25FluxErr,uAp9Flux,uAp9FluxErr,uExtendedness,uBlendedness,uFwhm,uCModelFlux,uCModelFluxErr,uHsmShapeRegauss_e1,uHsmShapeRegauss_e2,uHsmShapeRegauss_flag,uInputCount,uCalib_psf_used,uPsfFlux_apCorr,uPsfFlux_apCorrErr,uPsfFlux_area,uPsfFlux_flag,uPsfFlux_flag_apCorr,uPsfFlux_flag_edge,uPsfFlux_flag_noGoodPixels,uGaapFlux_Psf_apCorr,uGaapFlux_Psf_apCorrErr,uGaapFlux_Optimal_apCorr,uGaapFlux_Optimal_apCorrErr,uGaapFlux_0_7_apCorr,uGaapFlux_0_7_apCorrErr,uGaapFlux_1_0_apCorr,uGaapFlux_1_0_apCorrErr,uGaapFlux_flag,uGaapFlux_flag_edge,uGaapFlux_flag_gaussianization,uGaapFlux_Optimal_flag_bigPsf,uGaapFlux_0_7_flag_bigPsf,uGaapFlux_1_0_flag_bigPsf,uKronFlux_apCorr,uKronFlux_apCorrErr,uKronFlux_flag,uKronFlux_flag_apCorr,uKronFlux_flag_bad_radius,uKronFlux_flag_bad_shape,uKronFlux_flag_bad_shape_no_psf,uKronFlux_flag_edge,uKronFlux_flag_no_fallback_radius,uKronFlux_flag_no_minimum_radius,uKronFlux_flag_small_radius,uKronFlux_flag_used_minimum_radius,uKronFlux_flag_used_psf_radius,uDeblend_nChild,uDeblend_skipped,uBlendedness_flag,uPixelFlags,uPixelFlags_bad,uPixelFlags_clipped,uPixelFlags_clippedCenter,uPixelFlags_cr,uPixelFlags_crCenter,uPixelFlags_edge,uPixelFlags_inexact_psf,uPixelFlags_inexact_psfCenter,uPixelFlags_interpolated,uPixelFlags_interpolatedCenter,uPixelFlags_offimage,uPixelFlags_saturated,uPixelFlags_saturatedCenter,uPixelFlags_sensor_edge,uPixelFlags_sensor_edgeCenter,uPixelFlags_suspect,uPixelFlags_suspectCenter,uExtendedness_flag,uInputCount_flag,uInputCount_flag_noInputs,uCalib_astrometry_used,uCalib_photometry_reserved,uCalib_photometry_used,uCalib_psf_candidate,uCalib_psf_reserved,uApFlux_flag,uApFlux_flag_apertureTruncated,uApFlux_flag_sincCoeffsTruncated,uCalibFlux_flag,uCalibFlux_flag_apertureTruncated,uCalibFlux_flag_sincCoeffsTruncated,uCentroid_flag_almostNoSecondDerivative,uCentroid_flag_edge,uCentroid_flag_noSecondDerivative,uCentroid_flag_notAtMaximum,uCentroid_flag_resetToPeak,uShapeRound_flag,uShapeRound_flag_no_pixels,uShapeRound_flag_not_contained,uShapeRound_flag_parent_source,uShapeRound_Flux,uShapeRound_x,uShapeRound_xx,uShapeRound_xy,uShapeRound_y,uShapeRound_yy,uPsfShape_flag,uPsfShape_flag_no_pixels,uPsfShape_flag_not_contained,uPsfShape_flag_parent_source,uShape_flag,uShape_flag_no_pixels,uShape_flag_not_contained,uShape_flag_parent_source,yRa,yDecl,yPsFlux,yPsFluxErr,yBdE1,yBdE2,yBdReB,yBdReD,yBdChi2,yBdFluxB,yBdFluxBErr,yBdFluxD,yBdFluxDErr,yGaapFlux_Psf,yGaapFluxErr_Psf,yGaapFlux_0_7,yGaapFluxErr_0_7,yGaapFlux_1_0,yGaapFluxErr_1_0,yGaapFlux_Optimal,yGaapFluxErr_Optimal,yIxx,yIyy,yIxy,yIxxPsf,yIyyPsf,yIxyPsf,yKronRad,yKronFlux,yKronFluxErr,yApFlux,yApFluxErr,yAp25Flux,yAp25FluxErr,yAp9Flux,yAp9FluxErr,yExtendedness,yBlendedness,yFwhm,yCModelFlux,yCModelFluxErr,yHsmShapeRegauss_e1,yHsmShapeRegauss_e2,yHsmShapeRegauss_flag,yInputCount,yCalib_psf_used,yPsfFlux_apCorr,yPsfFlux_apCorrErr,yPsfFlux_area,yPsfFlux_flag,yPsfFlux_flag_apCorr,yPsfFlux_flag_edge,yPsfFlux_flag_noGoodPixels,yGaapFlux_Psf_apCorr,yGaapFlux_Psf_apCorrErr,yGaapFlux_Optimal_apCorr,yGaapFlux_Optimal_apCorrErr,yGaapFlux_0_7_apCorr,yGaapFlux_0_7_apCorrErr,yGaapFlux_1_0_apCorr,yGaapFlux_1_0_apCorrErr,yGaapFlux_flag,yGaapFlux_flag_edge,yGaapFlux_flag_gaussianization,yGaapFlux_Optimal_flag_bigPsf,yGaapFlux_0_7_flag_bigPsf,yGaapFlux_1_0_flag_bigPsf,yKronFlux_apCorr,yKronFlux_apCorrErr

In [5]:
df = butlerRFC.get('objectTable', tract=3828, patch=5, skymap='DC2')
print("%s rows and %s columns" % (df.shape))
df.tail()

28948 rows and 851 columns


column,detect_isIsolated,detect_isPrimary,refBand,coord_dec,detect_fromBlend,xErr,y,objectId,detect_isDeblendedModelSource,parentObjectId,refExtendedness,detect_isPatchInner,merge_peak_sky,detect_isTractInner,refFwhm,coord_ra,xy_flag,detect_isDeblendedSource,yErr,x,skymap,tract,patch,g_ra,g_decl,g_psfFlux,g_psfFluxErr,g_bdE1,g_bdE2,g_bdReB,g_bdReD,g_bdChi2,g_bdFluxB,g_bdFluxBErr,g_bdFluxD,g_bdFluxDErr,g_gaapFlux_Psf,g_gaapFluxErr_Psf,g_gaapFlux_0_7,g_gaapFluxErr_0_7,g_gaapFlux_1_0,g_gaapFluxErr_1_0,g_gaapFlux_Optimal,g_gaapFluxErr_Optimal,g_ixx,g_iyy,g_ixy,g_ixxPSF,g_iyyPSF,g_ixyPSF,g_kronRad,g_kronFlux,g_kronFluxErr,g_apFlux,g_apFluxErr,g_ap03Flux,g_ap03FluxErr,g_ap06Flux,g_ap06FluxErr,g_ap09Flux,g_ap09FluxErr,g_ap25Flux,g_ap25FluxErr,g_ap50Flux,g_ap50FluxErr,g_ap70Flux,g_ap70FluxErr,g_extendedness,g_blendedness,g_fwhm,g_cModelFlux,g_cModelFluxErr,g_hsmShapeRegauss_e1,g_hsmShapeRegauss_e2,g_hsmShapeRegauss_flag,g_inputCount,g_calib_psf_used,g_psfFlux_area,g_psfFlux_flag,g_psfFlux_flag_apCorr,g_psfFlux_flag_edge,g_psfFlux_flag_noGoodPixels,g_gaapFlux_flag,g_gaapFlux_flag_edge,g_gaapFlux_flag_gaussianization,g_gaapFlux_Optimal_flag_bigPsf,g_gaapFlux_0_7_flag_bigPsf,g_gaapFlux_1_0_flag_bigPsf,g_apFlux_3_0_flag,g_apFlux_6_0_flag,g_apFlux_9_0_flag,g_apFlux_25_0_flag,g_apFlux_50_0_flag,g_apFlux_70_0_flag,g_deblend_nChild,g_deblend_skipped,g_blendedness_flag,g_pixelFlags_bad,g_pixelFlags_clipped,g_pixelFlags_clippedCenter,g_pixelFlags_cr,g_pixelFlags_crCenter,g_pixelFlags_edge,g_pixelFlags_inexact_psf,g_pixelFlags_inexact_psfCenter,g_pixelFlags_interpolated,g_pixelFlags_interpolatedCenter,g_pixelFlags_offimage,g_pixelFlags_saturated,g_pixelFlags_saturatedCenter,g_pixelFlags_sensor_edge,g_pixelFlags_sensor_edgeCenter,g_pixelFlags_suspect,g_pixelFlags_suspectCenter,g_extendedness_flag,g_inputCount_flag,g_inputCount_flag_noInputs,g_calib_astrometry_used,g_calib_photometry_reserved,g_calib_photometry_used,g_calib_psf_candidate,g_calib_psf_reserved,g_apFlux_flag,g_apFlux_flag_apertureTruncated,g_apFlux_flag_sincCoeffsTruncated,g_calibFlux_flag,g_calibFlux_flag_apertureTruncated,g_calibFlux_flag_sincCoeffsTruncated,g_centroid_flag_almostNoSecondDerivative,g_centroid_flag_edge,g_centroid_flag_noSecondDerivative,g_centroid_flag_notAtMaximum,g_centroid_flag_resetToPeak,g_shapeRound_flag,g_shapeRound_flag_no_pixels,g_shapeRound_flag_not_contained,g_shapeRound_flag_parent_source,g_shapeRound_Flux,g_shapeRound_x,g_shapeRound_xx,g_shapeRound_xy,g_shapeRound_y,g_shapeRound_yy,g_psfShape_flag,g_psfShape_flag_no_pixels,g_psfShape_flag_not_contained,g_psfShape_flag_parent_source,g_shape_flag,g_shape_flag_no_pixels,g_shape_flag_not_contained,g_shape_flag_parent_source,g_kronFlux_flag,g_kronFlux_flag_bad_radius,g_kronFlux_flag_bad_shape,g_kronFlux_flag_bad_shape_no_psf,g_kronFlux_flag_edge,g_kronFlux_flag_no_fallback_radius,g_kronFlux_flag_no_minimum_radius,g_kronFlux_flag_small_radius,g_kronFlux_flag_used_minimum_radius,g_kronFlux_flag_used_psf_radius,y_ra,y_decl,y_psfFlux,y_psfFluxErr,y_bdE1,y_bdE2,y_bdReB,y_bdReD,y_bdChi2,y_bdFluxB,y_bdFluxBErr,y_bdFluxD,y_bdFluxDErr,y_gaapFlux_Psf,y_gaapFluxErr_Psf,y_gaapFlux_0_7,y_gaapFluxErr_0_7,y_gaapFlux_1_0,y_gaapFluxErr_1_0,y_gaapFlux_Optimal,y_gaapFluxErr_Optimal,y_ixx,y_iyy,y_ixy,y_ixxPSF,y_iyyPSF,y_ixyPSF,y_kronRad,y_kronFlux,y_kronFluxErr,y_apFlux,y_apFluxErr,y_ap03Flux,y_ap03FluxErr,y_ap06Flux,y_ap06FluxErr,y_ap09Flux,y_ap09FluxErr,y_ap25Flux,y_ap25FluxErr,y_ap50Flux,y_ap50FluxErr,y_ap70Flux,y_ap70FluxErr,y_extendedness,y_blendedness,y_fwhm,y_cModelFlux,y_cModelFluxErr,y_hsmShapeRegauss_e1,y_hsmShapeRegauss_e2,y_hsmShapeRegauss_flag,y_inputCount,y_calib_psf_used,y_psfFlux_area,y_psfFlux_flag,y_psfFlux_flag_apCorr,y_psfFlux_flag_edge,y_psfFlux_flag_noGoodPixels,y_gaapFlux_flag,y_gaapFlux_flag_edge,y_gaapFlux_flag_gaussianization,y_gaapFlux_Optimal_flag_bigPsf,y_gaapFlux_0_7_flag_bigPsf,y_gaapFlux_1_0_flag_bigPsf,y_apFlux_3_0_flag,y_apFlux_6_0_flag,y_apFlux_9_0_flag,y_apFlux_25_0_flag,y_apFlux_50_0_flag,y

In [6]:
# Current Columns
# Annoying long for a github render
# cols = butlerRFC.get('objectTable.columns', tract=3828, patch=5, skymap='DC2')
# sortedcols = [x for _, x in sorted(zip([c.lower() for c in cols], cols))]
# print('\n'.join(sortedcols))

In [7]:
# Proposed  Columns
# cols = butlerRFC.get('objectTable.columns', tract=3828, patch=5, skymap='DC2')
# sortedcols = [x for _, x in sorted(zip([c.lower() for c in cols], cols))]
# print('\n'.join(sortedcols))

## The Visit and CcdVisit Table

### Visit

* **dimensions:** None, it's global. 
* **specification:**  https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/postprocess.py#L1375
* **what**:  Visits that went into the survey
* **primary key**: `visitId`

*Questions:*
* Do you expect 'i' or  'i_sim_1.4' when you request `filterName`? Can remove `filterName` from the schema and replace with `physicalFilter` and `band`?
* Overall consistency: (`visit`, `detector`, `tract`, `patch`) vs. (`visitId`, `detectorId`, `tractId`, `patchId`)?
    > - I like Id <br>
    > - I like less typing <br>
    > - These were just keys to other tables. "Id" hints to users that this is a non-human-reabible database key. <br>
    > - Even the butler registry just uses tract/patch/visit

*TODO:* 
* This it the one case its a human readble number and a primary key. Either make `visit` the index or add visit to column list

*On DM-31825:*
* Made `visitId` index

*On RFC:*
* proposal to replace "filterName" with "physicalFilter" and "band"


In [8]:
df = butlerRFC.get('visitTable')
print("AFTER: %s rows and %s columns" % (df.shape))
df.tail()

AFTER: 345 rows and 11 columns


,filterName,band,ra,decl,skyRotation,azimuth,altitude,zenithDistance,airmass,obsStart,expTime
visitId,,,,,,,,,,,
227976,i_sim_1.4,i,58.304442,-35.810894,27.331870,132.645178,81.449196,8.550804,1.011402,2022-11-04 05:11:43.977800,30.0
192355,i_sim_1.4,i,56.286224,-35.023077,167.851118,219.434560,83.696415,6.303585,1.005309,2022-09-15 09:10:40.761800,30.0
227984,i_sim_1.4,i,57.337793,-37.254919,41.001905,148.743656,81.670495,8.329505,1.010585,2022-11-04 05:17:10.742200,30.0
193111,r_sim_1.4,r,59.536909,-34.593129,25.963643,109.701805,72.595876,17.404124,1.047824,2022-09-17 07:36:29.942200,30.0
397278,i_sim_1.4,i,53.783262,-37.516703,351.083244,114.430642,45.101554,44.898446,1.392382,2023-07-17 09:02:10.915100,30.0


### CcdVisit

* **dimensions:** None, it's global. 
* **specification:**  https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/postprocess.py#L1375
* **what**: One row per detector.
* **primary key**: `ccdVisitId`


**Questions:**
* Do we still want to call this the `CcdVisit` Table now that we refer to them as detectors everywhere?
     > JFB: in the butler we decided to not have this table. Notice how close all the values are. Let's run the experiment and see if delegates actually use this table.
* Eventually, there should be another table that has one row per snap not one row per visit? 
  - https://github.com/lsst/sdm_schemas/blob/master/yml/baselineSchema.yaml#L5077 has a `Visit_To_RawExposure`,  `RawExposure`, `RawCcdExposure`, `RawAmpExposure` (Related to topic at last DMLT. Out of scope for today)


**On DM-31825:**
* Made `ccdVisitId` the index. 
* Added `detector` and `band`.


In [9]:
df = butlerRFC.get('CcdVisitTable')
print("AFTER %s rows and %s columns" % (df.shape))
df.tail()

AFTER 11461 rows and 27 columns


,visitId,filterName,band,ra,decl,zenithDistance,zeroPoint,psfSigma,skyBg,skyNoise,detector,seeing,skyRotation,expMidpt,expTime,obsStart,darkTime,xSize,ySize,llcra,llcdec,ulcra,ulcdec,urcra,urcdec,lrcra,lrcdec
ccdVisitId,,,,,,,,,,,,,,,,,,,,,,,,,,,
397278076,397278,i_sim_1.4,i,55.534134,-37.281364,46.439495,31.834473,2.261530,1632.980957,48.643200,76,1.063932,351.083244,2023-07-17 09:02:10.915100,30.0,2023-07-17 09:01:55.915100,30.0,4071,3999,55.698544,-37.370724,55.418725,-37.410502,55.370052,-37.191759,55.649096,-37.152152
397278078,397278,i_sim_1.4,i,55.771524,-37.008958,46.668964,31.833731,2.255837,1638.633667,48.802895,78,1.060634,351.083244,2023-07-17 09:02:10.915100,30.0,2023-07-17 09:01:55.915100,30.0,4071,3999,55.935534,-37.097993,55.657070,-37.138241,55.607825,-36.919679,55.885527,-36.879618
397278117,397278,i_sim_1.4,i,55.718776,-36.777089,46.664692,31.833515,2.244721,1638.865601,48.853016,117,1.055218,351.083244,2023-07-17 09:02:10.915100,30.0,2023-07-17 09:01:55.915100,30.0,4071,3999,55.882277,-36.866191,55.604632,-36.906282,55.555592,-36.687753,55.832462,-36.647850
397278120,397278,i_sim_1.4,i,55.667221,-36.546521,46.661808,31.830772,2.250240,1639.317749,49.074760,120,1.057677,351.083244,2023-07-17 09:02:10.915100,30.0,2023-07-17 09:01:55.915100,30.0,4071,3999,55.830179,-36.635700,55.553355,-36.675592,55.504574,-36.457119,55.780627,-36.417416
397278123,397278,i_sim_1.4,i,55.615085,-36.315946,46.659122,31.836454,2.247108,1640.981445,48.785583,123,1.055853,351.083244,2023-07-17 09:02:10.915100,30.0,2023-07-17 09:01:55.915100,30.0,4071,3999,55.777488,-36.405220,55.501470,-36.444860,55.453005,-36.226458,55.728246,-36.187007


#  DIA Tables

* $\color{red}{\text{General Question}}$: The DIASourceTables pack flags into one column. Object, Source, ForcedSource tables that have boolean columns. Should we be consistent in the Parquet outputs from DRP in DP0.2?

> Consult with DAX <br>
> science users just wnat consistency <br>
> Punt for later. <br>

> bit-packed flags take less space, and APDB needs to be small  <br>
> DIA Source from DRP doesn't have to match APDB schema  <br>
> Parquet does have a 1-bit columns.  <br>


## The Raw DIASource Table
* **dimensions**: `visit`
* **Specification:** https://github.com/lsst/ap_association/blob/master/data/DiaSource.yaml
* **what:** `diaSourceTable` is just an aggregated `goodSeeingDiff_diaSrcTable`. It has not yet been associated into Objects. Only purpose is QA. See next.

**Questions**:
* We're not going to bother ingesting this, but do you anyone will ever want to fetch all the DiaSources from a whole visit for QA without its `diaObjectId`?


In [10]:
df = butler.get('diaSourceTable', visit=212071)
print("The table is %s rows and %s columns" % (df.shape))
df.tail()

The table is 6629 rows and 36 columns


,diaSourceId,ccdVisitId,filterName,diaObjectId,parentDiaSourceId,midPointTai,pixelId,bboxSize,flags,ra,decl,x,y,xErr,yErr,apFlux,apFluxErr,snr,psFlux,psFluxErr,dipMeanFlux,dipMeanFluxErr,dipFluxDiff,dipFluxDiffErr,dipLength,dipAngle,dipChi2,isDipole,totFlux,totFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF
76,113854797886521557,212071087,r,0,0,59867.214924,0,20,25165824,58.845664,-37.006881,1534.840074,3789.459778,NaN,NaN,131.512684,716.783426,0.183476,-49.576254,345.578937,4698.839573,NaN,81.057293,NaN,0.027158,-11.517111,2353.738494,True,68449.559944,331.678988,NaN,NaN,NaN,0.110835,-0.001505,-0.001505
77,113854797886521558,212071087,r,0,0,59867.214924,0,15,0,58.865078,-37.034306,967.430847,3791.172119,NaN,NaN,1360.589436,679.561534,2.002158,1096.246765,206.938536,1584.698964,NaN,1074.240524,NaN,NaN,NaN,NaN,False,996.665020,203.484197,0.066791,0.067258,-0.024602,0.110820,-0.001505,-0.001505
78,113854797886521559,212071087,r,0,0,59867.214924,0,14,0,58.883882,-37.051621,563.073120,3874.134033,NaN,NaN,-777.598075,675.877665,-1.150501,1119.937813,208.262442,1233.552889,NaN,649.370080,NaN,NaN,NaN,NaN,False,1141.266538,204.055555,0.104554,0.074490,-0.016670,0.110809,-0.001505,-0.001505
79,113854797886521560,212071087,r,0,0,59867.214924,0,19,25165824,58.864391,-37.021208,1177.960658,3897.962237,NaN,NaN,505.765540,686.469884,0.736763,-235.646828,268.511643,2688.509379,NaN,112.457213,NaN,0.040057,-18.615914,2329.710710,True,26909.450563,259.175180,0.003331,0.003331,0.000000,0.110825,-0.001505,-0.001505
80,113854797886521561,212071087,r,0,0,59867.214924,0,23,25165824,58.839842,-36.979575,2004.641928,3956.943040,NaN,NaN,-3548.747000,953.385905,-3.722260,-2486.878569,781.782987,9214.587788,NaN,-3472.609729,NaN,0.008682,-133.796856,8247.339609,True,509935.669325,758.067813,NaN,NaN,NaN,0.110847,-0.001505,-0.001505


## The DIASource Table

* **dimensions:** tract, patch
* **specification:** https://github.com/lsst/ap_association/blob/master/data/DiaSource.yaml
* **what**: DiaSource Table with diaObjectId for matching to the diaObject Table. 
* **primary key**: `diaSourceId`, foreign keys: (`diaObjectId`, `ccdVisitId`)

TO DO:
* This will be aggregated into a per-tract table. 

In [11]:
df = butler.get('goodSeeingDiff_assocDiaSrcTable', tract=3828, patch=3)
print("BEFORE: The table is %s rows and %s columns" % (df.shape))
df.tail()

BEFORE: The table is 5035 rows and 36 columns


,ccdVisitId,diaSourceId,filterName,diaObjectId,parentDiaSourceId,midPointTai,pixelId,bboxSize,flags,ra,decl,x,y,xErr,yErr,apFlux,apFluxErr,snr,psFlux,psFluxErr,dipMeanFlux,dipMeanFluxErr,dipFluxDiff,dipFluxDiffErr,dipLength,dipAngle,dipChi2,isDipole,totFlux,totFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF
5030,183772019,98661851440611484,g,3299854297281331488,0,59823.289126,0,39,25296896,56.601205,-37.075319,20.609337,1929.650269,None,None,NaN,NaN,NaN,NaN,NaN,2028.549295,NaN,2252.508387,NaN,NaN,NaN,NaN,False,930.324334,112.728001,NaN,NaN,NaN,0.243043,0.002566,0.002566
5031,183772019,98661851440611490,g,3299854297281331250,0,59823.289126,0,50,25165824,56.520353,-37.067885,1118.337152,2332.342612,None,None,681.572084,531.572514,1.282181,-4473.353070,579.680867,9457.864009,NaN,2981.951716,NaN,0.012755,-172.065993,1490.119660,True,301622.332512,540.647869,NaN,NaN,NaN,0.243036,0.002565,0.002565
5032,183772019,98661851440611494,g,3299854297281331427,0,59823.289126,0,19,25165824,56.572759,-37.041657,275.099487,2615.986328,None,None,-56.902023,276.075083,-0.206111,-827.541583,166.343832,1149.238586,NaN,-231.847228,NaN,NaN,NaN,NaN,False,14228.197418,157.645654,NaN,NaN,NaN,0.243081,0.002566,0.002566
5033,183772019,98661851440611495,g,3299854297281331227,0,59823.289126,0,54,25165824,56.554277,-37.036235,510.348794,2773.352172,None,None,-1504.073365,580.622432,-2.590450,-13597.210369,624.756783,15820.001150,NaN,3290.369566,NaN,0.448537,1.796040,254775.958747,True,358331.820575,583.893961,NaN,NaN,NaN,0.243084,0.002566,0.002566
5034,183772019,98661851440611496,g,3299854297281331229,0,59823.289126,0,36,25165824,56.576823,-37.031881,176.876216,2773.737787,None,None,156.346117,416.745821,0.375159,-1689.587923,418.871827,5381.885902,NaN,686.560428,NaN,0.021333,162.958117,728.219295,True,157833.128688,398.369401,NaN,NaN,NaN,0.243095,0.002566,0.002566


In [12]:
butlerRFC = dafButler.Butler('/repo/dc2', collections=['u/yusra/dc2/DM-31825'])
df = butlerRFC.get('goodSeeingDiff_assocDiaSrcTable', tract=3828, patch=3)
print("AFTER: The table is %s rows and %s columns" % (df.shape))
df.tail()

AFTER: The table is 5035 rows and 66 columns


,ccdVisitId,filterName,diaObjectId,parentDiaSourceId,midPointTai,pixelId,bboxSize,flags,ra,decl,x,y,xErr,yErr,apFlux,apFluxErr,snr,psFlux,psFluxErr,dipMeanFlux,dipMeanFluxErr,dipFluxDiff,dipFluxDiffErr,dipLength,dipAngle,dipChi2,isDipole,totFlux,totFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,base_PixelFlags_flag,base_PixelFlags_flag_offimage,base_PixelFlags_flag_edge,base_PixelFlags_flag_interpolated,base_PixelFlags_flag_saturated,base_PixelFlags_flag_cr,base_PixelFlags_flag_bad,base_PixelFlags_flag_suspect,base_PixelFlags_flag_interpolatedCenter,base_PixelFlags_flag_saturatedCenter,base_PixelFlags_flag_crCenter,base_PixelFlags_flag_suspectCenter,slot_Centroid_flag,slot_Centroid_pos_flag,slot_Centroid_neg_flag,slot_ApFlux_flag,slot_ApFlux_flag_apertureTruncated,slot_PsfFlux_flag,slot_PsfFlux_flag_noGoodPixels,slot_PsfFlux_flag_edge,ip_diffim_forced_PsfFlux_flag,ip_diffim_forced_PsfFlux_flag_noGoodPixels,ip_diffim_forced_PsfFlux_flag_edge,slot_Shape_flag,slot_Shape_flag_unweightedBad,slot_Shape_flag_unweighted,slot_Shape_flag_shift,slot_Shape_flag_maxIter,slot_Shape_flag_psf,coord_ra,coord_dec
diaSourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
114213463089217720,212739153,r,3299854297281331812,0,59868.145249,0,15,25165824,56.679397,-37.066777,4016.036377,1430.046753,None,None,-907.810823,342.023950,-2.654232,-482.731509,90.655158,738.691020,NaN,-7.388277e+02,NaN,NaN,NaN,NaN,False,-459.076523,104.237992,NaN,NaN,NaN,0.092322,-0.002621,-0.002621,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,56.679397,-37.066777
98661851440611457,183772019,g,3299854297281331483,0,59823.289126,0,131,41943836,56.519363,-37.155467,1502.927856,798.959106,None,None,386466.219961,744.655619,522.119685,139760.017992,398.936490,534375.347670,NaN,1.046783e+06,NaN,NaN,NaN,NaN,False,211798.645443,408.203637,0.234595,0.381552,0.078326,0.242920,0.002564,0.002564,False,False,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,56.519363,-37.155467
98661851440611460,183772019,g,3299854297281331484,0,59823.289126,0,18,109051904,56.515179,-37.153511,1553.032104,847.324280,None,None,1065.420353,251.047382,4.243903,564.400390,110.287294,809.391706,NaN,5.980503e+02,NaN,NaN,NaN,NaN,False,725.489558,112.313378,69.680890,100.364756,78.789729,0.242921,0.002564,0.002564,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,56.515179,-37.153511
98661851440611466,183772019,g,3299854297281331485,0,59823.289126,0,142,41943836,56.558554,-37.122372,815.715742,1247.929137,None,None,329271.224789,760.147128,434.984489,133939.071367,401.248567,550119.044558,NaN,1.049796e+06,NaN,2.568636,8.139537,205143.717937,True,215316.427934,410.370964,0.222360,0.379973,-0.043119,0.242972,0.002565,0.002565,False,False,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,56.558554,-37.122372
98661851440611484,183772019,g,3299854297281331488,0,59823.289126,0,39,25296896,56.601205,-37.075319,20.609337,1929.650269,None,None,NaN,NaN,NaN,NaN,NaN,2028.549295,NaN,2.252508e+03,NaN,NaN,NaN,NaN,False,930.324334,112.728001,NaN,NaN,NaN,0.243043,0.002566,0.002566,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,56.601205,-37.075319


## The DIAObject Table

* **dimensions:** tract, patch
* **specification:** https://github.com/lsst/meas_base/blob/3bd782b489b14a3f46934e5e0dd43256f727c0fe/python/lsst/meas/base/diaCalculationPlugins.py
* **what**:  `DiaObjects` with aggregate metrics computes on associated `DiaSources`.
* **primary key**: `diaObjectId`, foreign keys: (`diaObjectId`, `ccdVisitId`)

**Questions:**
* 


In [13]:
df = butler.get('goodSeeingDiff_diaObjTable', tract=3828, patch=3)
print("BEFORE: The table is %s rows and %s columns" % (df.shape))
df.head()

BEFORE: The table is 1377 rows and 4 columns


,diaObjectId,ra,decl,nDiaSources
0,3299854297281331201,56.645501,-37.012695,53
1,3299854297281331202,56.631406,-37.019004,18
2,3299854297281331203,56.670294,-37.021876,74
3,3299854297281331204,56.513706,-37.105570,1
4,3299854297281331205,56.528482,-37.179682,1


In [14]:
df = butlerRFC.get('goodSeeingDiff_diaObjTable', tract=3828, patch=3)
print("AFTER: The table is %s rows and %s columns" % (df.shape))
print(df.head())
print(df.columns)

AFTER: The table is 1377 rows and 3 columns
                            ra       decl  nDiaSources
diaObjectId                                           
3299854297281331201  56.645501 -37.012695           53
3299854297281331202  56.631406 -37.019004           18
3299854297281331203  56.670294 -37.021876           74
3299854297281331204  56.513706 -37.105570            1
3299854297281331205  56.528482 -37.179682            1
Index(['ra', 'decl', 'nDiaSources'], dtype='object')


# ForcedSource Table

The plan is to associate the `DiaObjects` and `CoaddSources` (what we are calling objects in the pipelines today) into final `Objects` before performing forced photometry on both the difference images and calexps. For DP0.2, we're proposing to skip this association step, so that users can choose whether they want to look at lightcurves of the sources detected on coadds, or the sources detected on difference images. 
* Pro: better for QA. Easier to answer questions like "where did this junk detection come from?" and to keep our very new DRP DC2 DIA pipeline separate from the more battle tested one. 
* Con: Users are going to think it's always going to be this way, and may get used to it. 



> - OK as long as we clearly explain to the delegates why we're doing it this way.  <br>
> - There's a lot that is different from how're we are going to use this in the future. So this won't be the only thing we tell them is changing in the future. OK  <br>
> - If this does lock us in to future data releases, is that bad? <br>
> - JFB: variable stars (EB: + AGN - which is good) will get measured multiple times.  <br>
> - Colin: just redundancy. OK <br>


## ForcedSource

* **dimensions:** tract
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/ForcedSource.yaml
* **what**: Object centroids (from `deepCoadd_ref`) forced photometered on calexps and diffims in one table.  
* **primary key**: `forcedSourceId` foreign keys: (`objectId`, `ccdVisitId`)

**Questions:**
* `band` or `filterName` on all?
* Includes patch overlaps on purpose so that users can use the overlaps for QA or remove using `detect_isPrimary` or `detect_isPatchInner` for analysis. OK?
* Includes all bands in one table. OK?


>  CS: I thought we'd remove the overlaps before ingest  <br>
>  JFB: I think people will want the overlaps to test how well we did the overlap resolution.   <br>
>  YA: the instructions for all table use is "USE detect_isPrimary!"  <br>

**TODO:**
* flags are all False: DM-31686



In [15]:
butlerForced = dafButler.Butler('/repo/dc2', collections=['u/yusra/DC2/old-bps/DM-28394'])
df = butlerForced.get('ForcedSourceTable_tract', tract=3828)
print("BEFORE The table is %s rows and %s columns" % (df.shape))
df.tail()

BEFORE The table is 4731729 rows and 35 columns


,detect_isPrimary,detect_isTractInner,detect_isPatchInner,objectId,parentObjectId,coord_ra,coord_dec,ccdVisitId,band,PsFlux,PsFluxErr,PsFluxFlag,psFluxApCorr,psFluxApCorrErr,PsDiffFlux,PsDiffFluxErr,PsDiffFluxFlag,psDiffFluxApCorr,psDiffFluxApCorrErr,localBackground_instFlux,localBackground_instFluxErr,pixelFlags,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_saturated,pixelFlags_cr,pixelFlags_bad,pixelFlags_suspect,pixelFlags_interpolatedCenter,pixelFlags_saturatedCenter,pixelFlags_crCenter,pixelFlags_suspectCenter,skymap,tract,patch
1649962333012797647,False,True,False,1649962333012797647,0,57.343880,-36.855717,228020115,i,76.166886,259.896934,False,0.979562,0.0,-71.921595,248.190933,False,0.979562,0.0,-1.525026,68.478883,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797649,False,True,True,1649962333012797649,0,57.351256,-36.849351,228020115,i,269.240720,260.213646,False,0.978902,0.0,265.512093,248.371402,False,0.978902,0.0,0.612889,71.841647,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797650,False,True,False,1649962333012797650,0,57.342839,-36.849067,228020115,i,-127.663248,258.736629,False,0.980635,0.0,-65.135659,247.298520,False,0.980635,0.0,-1.243882,70.214145,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797653,False,True,True,1649962333012797653,0,57.362364,-36.840114,228020115,i,427.288634,259.833468,True,0.977889,0.0,NaN,NaN,True,0.977889,0.0,2.321080,67.406001,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797655,False,True,True,1649962333012797655,0,57.349442,-36.833464,228020115,i,-326.638206,258.624514,False,0.981509,0.0,-312.112720,245.698169,False,0.981509,0.0,-1.556618,69.416625,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7


In [16]:
butlerForcedRFC = dafButler.Butler('/repo/dc2', collections=['u/yusra/dc2/forcedPhot-DM-31825'])
df = butlerForcedRFC.get('ForcedSourceTable_tract', tract=3828)
print("AFTER: The table is %s rows and %s columns" % (df.shape))
df.head()

AFTER: The table is 4731729 rows and 43 columns


,objectId,parentObjectId,coord_ra,coord_dec,ccdVisitId,band,psFlux,psFluxErr,psFluxFlag,psFluxApCorr,psFluxApCorrErr,psDiffFlux,psDiffFluxErr,psDiffFluxFlag,psDiffFluxApCorr,psDiffFluxApCorrErr,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,localBackground_instFlux,localBackground_instFluxErr,pixelFlags,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_saturated,pixelFlags_cr,pixelFlags_bad,pixelFlags_suspect,pixelFlags_interpolatedCenter,pixelFlags_saturatedCenter,pixelFlags_crCenter,pixelFlags_suspectCenter,skymap,tract,patch,detect_isPrimary,detect_isTractInner,detect_isPatchInner
forcedSourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
260939781364514817,1649909556454626711,0,57.064046,-37.077514,243019109,z,901.235547,466.096054,True,0.997542,0.0,NaN,NaN,True,0.997542,0.0,0.944972,False,0.000228,False,-3.332194e-07,-9.082098e-07,-3.344389e-07,9.092316e-07,-1.413303,64.947961,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,1,False,True,False
260939781364514818,1649909556454626720,0,57.061050,-37.078101,243019109,z,1223.996114,465.648345,True,0.997946,0.0,NaN,NaN,True,0.997946,0.0,0.944972,False,0.000228,False,-3.331884e-07,-9.082134e-07,-3.344098e-07,9.092395e-07,0.297157,68.879581,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,1,False,True,False
260939781364514819,1649909556454626769,0,57.062155,-37.076319,243019109,z,596.634041,464.831217,True,0.997781,0.0,51.996148,505.869793,True,0.997781,0.0,0.944972,False,0.000228,False,-3.332021e-07,-9.082135e-07,-3.344198e-07,9.092384e-07,-2.734717,65.325442,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,1,False,True,False
260939781364514820,1649909556454626803,0,57.063587,-37.075622,243019109,z,509.089282,465.725285,False,0.997586,0.0,161.259862,504.148075,False,0.997586,0.0,0.944972,False,0.000228,False,-3.332176e-07,-9.082121e-07,-3.344335e-07,9.092351e-07,-2.336763,65.483577,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,1,False,True,False
260939781364514821,1649909556454626819,0,57.062788,-37.075246,243019109,z,-17.361036,464.075591,False,0.997687,0.0,-342.051044,505.877315,False,0.997687,0.0,0.944972,False,0.000228,False,-3.332101e-07,-9.082136e-07,-3.344255e-07,9.092379e-07,-0.695348,66.210565,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,1,False,True,False


## ForcedSourceOnDiaObjects Table

* **dimensions:** tract
* **specification:** https://github.com/lsst/obs_lsst/blob/master/pipelines/imsim/DRP.yaml
* **what**: diaObject centroids (from `goodSeeingDiff_diaObjTable`) forced photometered on calexps and diffims in one table.  
* **primary key**:  `forcedSourceOnDiaObjectId` with foreign keys: (`diaObjectId`, `ccdVisitId`)

**Questions**:
* There is no `detect_isPrimary` or `detect_isPatchInner` because the reference table is matched diaSources (were detected on per-detector diffims NOT a patch. No overlaps. But should I geometrically add a `detect_isTractInner` on diaObject? 

> - JFB: We should look into use of inner bbox patches to make sure its deterministic and we're not losing or duplicating sources. <br>
> - Discussion concludes yes, add `detect_isTractInner`


Work in progress on DM-31765.  


In [17]:
butlerForcedRFC = dafButler.Butler('/repo/dc2', collections=['u/yusra/DC2/w32/DM-31601'])
df = butlerForcedRFC.get('ForcedSourceOnDiaObjectTable', tract=3828, patch=10)
print("The table is %s rows and %s columns (only 1 patch). Ignore the NaNs for now." % (df.shape))
df.head()

The table is 5326 rows and 32 columns (only 1 patch). Ignore the NaNs for now.


,diaObjectId,parentObjectId,coord_ra,coord_dec,ccdVisitId,band,PsFlux,PsFluxErr,PsFluxFlag,psFluxApCorr,psFluxApCorrErr,PsDiffFlux,PsDiffFluxErr,PsDiffFluxFlag,psDiffFluxApCorr,psDiffFluxApCorrErr,localBackground_instFlux,localBackground_instFluxErr,pixelFlags,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_saturated,pixelFlags_cr,pixelFlags_bad,pixelFlags_suspect,pixelFlags_interpolatedCenter,pixelFlags_saturatedCenter,pixelFlags_crCenter,pixelFlags_suspectCenter,skymap,tract,patch
diaObjectId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3299977442583642157,3299977442583642157,0,56.724899,-36.787532,189382138,y,NaN,NaN,False,0.994731,0.0,NaN,NaN,False,0.994731,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC2,3828,10
3299977442583642196,3299977442583642196,0,56.640239,-36.799048,189382138,y,NaN,NaN,False,0.994084,0.0,NaN,NaN,False,0.994084,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC2,3828,10
3299977442583642203,3299977442583642203,0,56.718453,-36.786014,189382138,y,NaN,NaN,False,0.994621,0.0,NaN,NaN,False,0.994621,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC2,3828,10
3299977442583642210,3299977442583642210,0,56.774919,-36.798061,189382138,y,NaN,NaN,False,0.994760,0.0,NaN,NaN,False,0.994760,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC2,3828,10
3299977442583642229,3299977442583642229,0,56.659937,-36.790960,189382138,y,NaN,NaN,False,0.994007,0.0,NaN,NaN,False,0.994007,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC2,3828,10


## Overall Consistency Questions
* DataFrames: Do we want Primary Key as index, column, or both?
    > - WTF?
    > - Does qserv care? 
    > - As long as it appears in the csv files, then its fine. 
    > - I think its weird for the index to be in the column list too. 
    > - YA: I'm moving forward with only Index
    
* Only floats that need to be doubles are ra/decl? All else can be singles. 

In [18]:
# Added After:
# Example of what I mean by Do we want Primary Key as index, column or both:

df = butlerRFC.get('goodSeeingDiff_diaObjTable', tract=3828, patch=3)
print("Primary Key: \n   The table is %s rows and %s columns" % (df.shape))
print(df.head())
print(df.columns)

df = df.reset_index(drop=False)
print("\nColumn \n   The table is %s rows and %s columns" % (df.shape))
print(df.head())
print(df.columns)

df = df.set_index("diaObjectId", drop=False)
print("\nBoth \n   The table is %s rows and %s columns" % (df.shape))
print(df.head())
print(df.columns)


Primary Key: 
   The table is 1377 rows and 3 columns
                            ra       decl  nDiaSources
diaObjectId                                           
3299854297281331201  56.645501 -37.012695           53
3299854297281331202  56.631406 -37.019004           18
3299854297281331203  56.670294 -37.021876           74
3299854297281331204  56.513706 -37.105570            1
3299854297281331205  56.528482 -37.179682            1
Index(['ra', 'decl', 'nDiaSources'], dtype='object')

Column 
   The table is 1377 rows and 4 columns
           diaObjectId         ra       decl  nDiaSources
0  3299854297281331201  56.645501 -37.012695           53
1  3299854297281331202  56.631406 -37.019004           18
2  3299854297281331203  56.670294 -37.021876           74
3  3299854297281331204  56.513706 -37.105570            1
4  3299854297281331205  56.528482 -37.179682            1
Index(['diaObjectId', 'ra', 'decl', 'nDiaSources'], dtype='object')

Both 
   The table is 1377 rows and 4 colu